Make a copy of this notebook and rename using your USERID in the following format, 2017CSZ8058

Give editor access to keshavkolluru@gmail.com, vishalsaley114@gmail.com and kartikeya.badola@gmail.com


In [1]:
## DONT CHANGE THIS CELL 
# this is currently the same as dev.data.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt

--2021-10-11 14:38:21--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt [following]
--2021-10-11 14:38:21--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.data.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63639 (62K) [text/plain]
Saving to: ‘test.data.txt’

test.data.txt       100%[===================>]  62.15K   116KB/s    in 0.5s    

2021-10-11 14:38:23 (116 KB/s) - ‘test.data.txt’ saved [63639/63639]



In [2]:
## Replace with the right link that contains the zip file uploaded from the training
!gdown https://drive.google.com/uc?id=1rsGmoqECZNDPNxcqM6e9IIejhRq-QT4m

Downloading...
From: https://drive.google.com/uc?id=1rsGmoqECZNDPNxcqM6e9IIejhRq-QT4m
To: /content/2018EE10434_A_model.zip
100% 25.1M/25.1M [00:00<00:00, 40.8MB/s]


In [3]:
!unzip 2018EE10434_A_model.zip

Archive:  2018EE10434_A_model.zip
   creating: MODEL/
  inflating: MODEL/vocab             
  inflating: MODEL/Model.pt          


In [4]:
!pip install -U torchtext==0.10.0

In [5]:
## Import relevant packages

import os

import time
import datetime

import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

import logging
from argparse import ArgumentParser

from pdb import set_trace
import pandas as pd
from torch.utils.data import Dataset
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, LabelField
from torchtext.data.utils import get_tokenizer
import spacy
# import torch
torch.manual_seed(4)
import random
random.seed(4)
import numpy as np
np.random.seed(4)
import torchtext
import dill

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
## YOUR testing code
def prepareDataset():
    x_test = pd.read_csv('/content/test.data.txt', sep = '\t',header = None, names = ["word","pos","P1P2" , "sentence1", "sentence2"])
    a = x_test["P1P2"].str.split(pat = "-",expand = True)
    a.columns = ['P1','P2']
    x_test = x_test.join(a)
    x_test.to_csv('TestData.csv')
prepareDataset()

In [7]:
spacy_tokens = spacy.load('en')

def tokenizer(sentence):
    return [token.text for token in spacy_tokens.tokenizer(sentence)]

with open(os.path.join("MODEL", "vocab"), 'rb') as f:
  sentence = dill.load(f)

position = Field(sequential=False,use_vocab=False)
fields = {'sentence1' : ('sentence1' , sentence), 'sentence2' : ('sentence2' , sentence), 'word' : ('word' , sentence) , 'P1' : ('P1',position), 'P2' : ('P2',position) }
test_data = TabularDataset.splits(path = './' , test = 'TestData.csv'  , format = 'csv' , fields= fields)
test_iter  = BucketIterator.splits(test_data, batch_size=32 , device = 'cpu' ,sort = False , repeat = False, shuffle = False , sort_key = None)[0]

In [8]:
class LSTMModel(nn.Module):
  def __init__(self, embed_dim, hidden, n_layers):
        super(LSTMModel, self).__init__()
        self.hidden = hidden
        self.n_layers = n_layers
        self.embed = nn.Embedding.from_pretrained(sentence.vocab.vectors, freeze=False)
        self.lstm = nn.LSTM(embed_dim, hidden,num_layers=n_layers,bidirectional=False)#batch_first = True #dropout=0.2
        
  def forward(self, input):
  
        input.sentence1 = input.sentence1.to(device)
        input.sentence2 = input.sentence2.to(device)
        input.P1 = input.P1.to(device)
        input.P2 = input.P2.to(device)
        embed_out = self.embed(input.sentence1).to(device)
        # embed_out = self.dropout(embed_out).to(device)
        hidden_in = torch.zeros(self.n_layers ,input.sentence1.shape[1], self.hidden).to(device)
        cell_in = torch.zeros( self.n_layers , input.sentence1.shape[1], self.hidden).to(device)
        out_lstm, (hidden_out, cell_out) = self.lstm(embed_out,(hidden_in, cell_in))
        
        embed_out2 = self.embed(input.sentence2).to(device)
        # embed_out2 = self.dropout(embed_out2).to(device)
        hidden_in2 = torch.zeros(self.n_layers  ,input.sentence2.shape[1], self.hidden).to(device)
        cell_in2 = torch.zeros( self.n_layers , input.sentence2.shape[1], self.hidden).to(device)
        out_lstm2, (hidden_out2, cell_out2) = self.lstm(embed_out2,(hidden_in2, cell_in2))
        
        dim_1 = out_lstm[input.P1,torch.arange(out_lstm.size(1))]
        dim_2 = out_lstm2[input.P2,torch.arange(out_lstm2.size(1))]
        
        cos = nn.CosineSimilarity(dim=1, eps=1e-8)
        output = cos(dim_1, dim_2)
        
        output = torch.sigmoid(output)

        return output

In [9]:
emb_dim = sentence.vocab.vectors.shape[1]
hidden = 512
layers = 2
model = LSTMModel(emb_dim,hidden,layers)

In [10]:
loadingModel = torch.load("/content/MODEL/Model.pt", map_location=device)

model.load_state_dict(loadingModel['model_dict'])
model = model.to(device)

model.eval()
test_iter.init_epoch()
answers = []

with torch.no_grad():
    for batch_idx, batch in enumerate(test_iter):
        answer = model(batch)
        answer = np.array((answer>=0.5).int())
        answer = answer.tolist()
        answers.append(answer)
        
    Answers = [ans for answer in answers for ans in answer]

output = pd.Series(Answers) 
output = output.apply(lambda x: 'F' if x == 0 else 'T')
output.to_csv("output.txt", header=None, index=None)

In [11]:
## DONT CHANGE THIS CELL
# Your testing code must produce a file output.txt with predictions as T and F in each line

## Final Evaluation 
# this is currently the same as dev.gold.txt
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
correct, total = 0., 0.
for prediction, gold in zip(open('output.txt'), open('test.gold.txt')):
  prediction, gold = prediction.strip(), gold.strip()
  total += 1
  if prediction == gold:
    correct += 1

## Report this as the final validation performance 
print('Performance = ', (correct/total))

--2021-10-11 14:39:09--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt [following]
--2021-10-11 14:39:10--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/test.gold.txt
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1276 (1.2K) [text/plain]
Saving to: ‘test.gold.txt’

test.gold.txt       100%[===================>]   1.25K  --.-KB/s    in 0s      

2021-10-11 14:39:11 (62.4 MB/s) - ‘test.gold.txt’ saved [1276/1276]

Performance =  0.6442006269592476
